In [1]:
import nltk
import os
import re
from textstat import textstat
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
import pandas as pd
output_df = pd.read_excel("Output Data Structure.xlsx")
df = pd.read_excel("input.xlsx", index_col="URL_ID")
df_URL = df["URL"]
new_df = pd.DataFrame()

In [3]:
my_file = open(r"C:\Users\Gaurav\BlackCoffers\MasterDictionary-20230901T191027Z-001\MasterDictionary\positive-words.txt", "r")
my_file2 = open(r"C:\Users\Gaurav\BlackCoffers\MasterDictionary-20230901T191027Z-001\MasterDictionary\negative-words.txt", "r")
data = my_file.read()
data2 = my_file2.read()
positive_words = data.split("\n")
negative_words = data2.split("\n")
my_file.close()
my_file2.close()

In [4]:
output_df.columns

Index(['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [5]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [6]:
new_df = pd.DataFrame(columns = ['URL_ID', 
                                 'URL', 
                                 'POSITIVE SCORE', 
                                 'NEGATIVE SCORE', 
                                 'POLARITY SCORE', 
                                 'SUBJECTIVITY SCORE', 
                                 'AVG SENTENCE LENGTH',
                                 'PERCENTAGE OF COMPLEX WORDS',
                                 'FOG INDEX',
                                 'AVG NUMBER OF WORDS PER SENTENCE',
                                 'COMPLEX WORD COUNT',
                                 'WORD COUNT',
                                 'SYLLABLE PER WORD',
                                 'PERSONAL PRONOUNS',
                                 'AVG WORD LENGTH'])
new_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH


In [7]:
def wordandSentance_length(file_name):
    my_new_file = open(r"C:\Users\Gaurav\BlackCoffers\new_saved_data\stemmed_words\{}".format(file_name), "r", encoding="utf-8")
    file_text = my_new_file.read()
    file_text.lower()
    word_tokens = word_tokenize(file_text)
    sent_tokens = sent_tokenize(file_text)
    my_new_file.close()
    return word_tokens, sent_tokens[0]

In [8]:
def syllable_count(data_retrieved):
    count = 0
    vowels = "aeiouy"
    if data_retrieved[0] in vowels:
        count += 1
    for index in range(1, len(data_retrieved)):
        if data_retrieved[index] in vowels and data_retrieved[index - 1] not in vowels:
            count += 1
    if data_retrieved.endswith("es" or "ed"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [24]:
def variable_calculation(data_retrieved, path, URL, new_df):
    count_pos = 0
    count_neg = 0
    for i in data_retrieved:
        if i in positive_words:
            count_pos += 1
    for i in data_retrieved:
        if i in negative_words:
            count_neg += 1
            
            
    polarity_score = (count_pos - count_neg)/ ((count_pos + count_neg) + 0.000001)
    
    
    subjectivity_score = (count_pos + count_neg)/ ((len(data_retrieved)) + 0.000001)
    
    
    word_tokens, sent_tokens = wordandSentance_length(path)
    avg_sentence_length = len(word_tokens)/len(sent_tokens.split("."))
    
    
    sentence = sent_tokens.split(".")
    words_in_sentence = 0
    for i in sentence:
        words_in_sentence += len(i.split(" "))
    avg_words_in_sentence = words_in_sentence/len(sentence)
    
    
    number_complex_words = textstat.polysyllabcount(sent_tokens)
    complex_word_percentage = (number_complex_words/len(word_tokens))*100
    
    
    fog_index = 0.4*(avg_sentence_length + complex_word_percentage)
    
    
    word_count = len(word_tokens)
    
    
    total_syllable_count = 0
    for i in data_retrieved:
        total_syllable_count += syllable_count(i)
    avg_syllable_word = total_syllable_count/len(word_tokens)
    
    
    pronounRegex = re.compile(r'\b(i|we|my|ours|(?-i:us))\b',re.I)
    pronouns = pronounRegex.findall(sent_tokens)
    
    
    overall_word_lengths = 0
    for i in word_tokens:
        overall_word_lengths += len(i)
    avg_word_lengths = overall_word_lengths/word_count
    
    
    return {'URL_ID' : float(path[0:-4]), 
                            'URL' : URL, 
                            'POSITIVE SCORE' : float(count_pos), 
                            'NEGATIVE SCORE': float(count_neg), 
                            'POLARITY SCORE': polarity_score,
                            'SUBJECTIVITY SCORE': subjectivity_score,
                            'AVG SENTENCE LENGTH': avg_sentence_length,
                            'PERCENTAGE OF COMPLEX WORDS': complex_word_percentage,
                            'FOG INDEX': fog_index,
                            'AVG NUMBER OF WORDS PER SENTENCE': avg_words_in_sentence,
                            'COMPLEX WORD COUNT':float(number_complex_words),
                            'WORD COUNT': float(word_count),
                            'SYLLABLE PER WORD':avg_syllable_word,
                            'PERSONAL PRONOUNS': float(len(pronouns)),
                            'AVG WORD LENGTH':avg_word_lengths }

In [10]:
def file_data_retrieval(file_path):
    my_file_new = open(r"C:\Users\Gaurav\BlackCoffers\new_saved_data\stemmed_words\{}".format(file_path), "r", encoding="utf-8")
    data_retrieved = my_file_new.read()
    data_retrieved = data_retrieved.strip()
    data_retrieved = data_retrieved.split(" ")
    my_file_new.close()
    return data_retrieved

In [22]:
dir_path = "C:/Users/Gaurav/BlackCoffers/new_saved_data/stemmed_words"
final_data = []
def file_path_retrieval():
    for path in os.listdir(dir_path):
        data_receieved = []
        data_receieved = file_data_retrieval(path)
        new_data = variable_calculation(data_receieved, path, df_URL[float(path[0:-4])], new_df)
        final_data.append(new_data)

In [25]:
file_path_retrieval()

In [27]:
new_df = pd.DataFrame(final_data)

In [28]:
result = output_df.set_index("URL_ID").combine_first(new_df.set_index("URL_ID"))

In [29]:
result

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
URL_ID,,,,,,,,,,,,,,
123.0,https://insights.blackcoffer.com/rise-of-telem...,118.0,24.0,0.661972,0.167453,35.333333,29.481132,25.925786,36.291667,250.0,848.0,2.909198,0.0,7.750000
321.0,https://insights.blackcoffer.com/rise-of-e-hea...,39.0,14.0,0.471698,0.186620,47.333333,41.901408,35.693897,48.166667,119.0,284.0,2.859155,0.0,7.943662
432.0,https://insights.blackcoffer.com/rise-of-telem...,46.0,30.0,0.210526,0.116386,65.400000,21.865443,34.906177,47.100000,143.0,654.0,2.571865,0.0,7.281346
2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,28.0,29.0,-0.017544,0.103825,45.750000,33.515483,31.706193,46.666667,184.0,549.0,2.500911,0.0,7.109290
2893.8,https://insights.blackcoffer.com/rise-of-chatb...,45.0,12.0,0.578947,0.096939,196.333333,2.716469,79.619921,18.666667,16.0,589.0,2.604414,2.0,7.232598
3355.6,https://insights.blackcoffer.com/rise-of-e-hea...,39.0,12.0,0.529412,0.088388,115.800000,9.844560,50.257824,34.800000,57.0,579.0,2.623489,0.0,7.405872
3817.4,https://insights.blackcoffer.com/how-does-mark...,60.0,6.0,0.818182,0.078385,281.000000,2.728351,113.491340,25.666667,23.0,843.0,2.568209,0.0,7.344009
4279.2,https://insights.blackcoffer.com/how-advertise...,7.0,0.0,1.000000,0.031963,109.500000,27.853881,54.941553,110.000000,61.0,219.0,2.369863,0.0,6.721461
4321.0,https://insights.blackcoffer.com/rise-of-telem...,46.0,30.0,0.210526,0.116386,65.400000,21.865443,34.906177,47.100000,143.0,654.0,2.571865,0.0,7.281346


In [31]:
result.to_excel("myfinalresult.xlsx")